In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd /content/drive/My\ Drive/Satellite\ Project-Village\ level

/content/drive/My Drive/Satellite Project-Village level


In [0]:
ls

deepsat-sat6/


In [0]:
cd deepsat-sat6/

/content/drive/My Drive/Satellite Project-Village level/deepsat-sat6


In [0]:
ls

ff.zip          sat6annotations.csv     X_test_sat6.csv   y_train_sat6.csv
mymodel.h5      sat-6-full.mat          X_train_sat6.csv
Run_Deepsat.sh  train_DeepSat_Model.py  y_test_sat6.csv


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [0]:
def load_data_and_labels(data, labels):
    data_df = pd.read_csv(data, header=None)
    X = data_df.values.reshape((-1,28,28,4)).clip(0,255).astype(np.uint8)
    labels_df = pd.read_csv(labels, header=None)
    Y = labels_df.values.getfield(dtype=np.int8)
    return X, Y
    
x_train, y_train = load_data_and_labels(data='X_train_sat6.csv',
                                        labels='y_train_sat6.csv')
x_test, y_test = load_data_and_labels(data='X_test_sat6.csv',
                                      labels='y_test_sat6.csv')

In [0]:
y_train.shape

(324000, 6)

In [0]:
x_train.shape

(324000, 28, 28, 4)

In [0]:
from tensorflow.keras import datasets, layers, models
from tensorflow import keras

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 4)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        1184      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 56,608
Trainable params: 56,608
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=200, epochs=6, verbose=1, validation_data=(x_test, y_test))

Train on 324000 samples, validate on 81000 samples
Epoch 1/6
324000/324000 [==============================] - 53s 163us/sample - loss: 0.1900 - acc: 0.9412 - val_loss: 0.1157 - val_acc: 0.9520
Epoch 2/6
324000/324000 [==============================] - 45s 139us/sample - loss: 0.0833 - acc: 0.9681 - val_loss: 0.0739 - val_acc: 0.9726
Epoch 3/6
324000/324000 [==============================] - 45s 139us/sample - loss: 0.0650 - acc: 0.9759 - val_loss: 0.0568 - val_acc: 0.9808
Epoch 4/6
324000/324000 [==============================] - 44s 137us/sample - loss: 0.0562 - acc: 0.9794 - val_loss: 0.0592 - val_acc: 0.9798
Epoch 5/6
324000/324000 [==============================] - 44s 137us/sample - loss: 0.0581 - acc: 0.9793 - val_loss: 0.0503 - val_acc: 0.9826
Epoch 6/6
324000/324000 [==============================] - 45s 138us/sample - loss: 0.0458 - acc: 0.9836 - val_loss: 0.0557 - val_acc: 0.9803


In [0]:
models.save_model(
    model,
    filepath = 'mymodel.h5'
)

In [0]:
## Making a tensorflow 2.0 resnet architecture

def res_net_block(input_data, filters, conv_size):
  x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Add()([x, input_data])
  x = layers.Activation('relu')(x)
  return x

In [0]:


inputs = keras.Input(shape = (28,28,4))
x = layers.Conv2D(32,3,activation = 'relu')(inputs)
x = layers.Conv2D(64,3,activation = 'relu')(x)
num_res_net_blocks = 10

for i in range(num_res_net_blocks):
  x = res_net_block(x,64,64)

x = layers.Conv2D(64,3,activation = 'relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(6,activation = 'softmax')(x)

res_net_model = keras.Model(inputs,outputs)


print(res_net_model.summary())



Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 28, 28, 4)]  0                                            
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, 26, 26, 32)   1184        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 24, 24, 64)   18496       conv2d_96[0][0]                  
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 24, 24, 64)   16777280    conv2d_97[0][0]                  
____________________________________________________________________________________________

In [0]:
res_net_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = res_net_model.fit(x_train, y_train, batch_size=64, epochs=6, verbose=1, validation_data=(x_test, y_test))

Train on 324000 samples, validate on 81000 samples
Epoch 1/6
 25024/324000 [=>............................] - ETA: 19:29:56 - loss: 0.3042 - acc: 0.9086